In [1]:
# !pip install surprise

In [2]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest.zip
!unzip ml-latest.zip

--2023-08-20 13:09:32--  https://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 350896731 (335M) [application/zip]
Saving to: ‘ml-latest.zip’

ml-latest.zip       100%[===================>] 334.64M   108MB/s    in 3.2s    

2023-08-20 13:09:36 (104 MB/s) - ‘ml-latest.zip’ saved [350896731/350896731]

Archive:  ml-latest.zip
   creating: ml-latest/
  inflating: ml-latest/tags.csv      
  inflating: ml-latest/links.csv     
  inflating: ml-latest/README.txt    
  inflating: ml-latest/ratings.csv   
  inflating: ml-latest/genome-tags.csv  
  inflating: ml-latest/genome-scores.csv  
  inflating: ml-latest/movies.csv    


In [3]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
import pandas as pd

In [4]:
movies = pd.read_csv('ml-latest/movies.csv')
ratings = pd.read_csv('ml-latest/ratings.csv').sample(frac=0.24)
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [5]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.movieId,
    'rating': movies_with_ratings.rating
})
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [7]:
from surprise import SVD

algo = SVD(n_factors=3, n_epochs=30, random_state=42,
           lr_bu=0.007, lr_bi=0.003, lr_pu=0.01, lr_qi=0.3,
           reg_bu=0.1, reg_bi=0.0001, reg_pu=0.1, reg_qi=1)
crossval = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True, n_jobs=-1)
crossval['test_rmse'].mean()

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8767  0.8757  0.8762  0.8770  0.8758  0.8763  0.0005  
Fit time          111.76  112.67  100.24  90.95   55.60   94.25   20.91   
Test time         54.18   42.45   33.38   39.88   22.54   38.48   10.43   


0.8763044681639787

Мне удалось лишь приблизиться к желаемому уровню метрики RMSE равному 0.87, но не достичь его. Наилучший показатель 0.876 после подбора гиперпараметров получился у SVD модели. Также, поскольку в бесплатной версии Колаба доступная память ограничена 12.7 Гб, я не смог обработать весь датасет, оставив только 24% данных таблицы ratings. Таблицу movies я оставил целиком. Хороший результат SVD в том числе обеспечен тем, что он эффективнее других алгоритмов расходует память в процессе обучения. Он смог принять в себя наибольший процент данных таблицы ratings и поэтому дал наилучшие предсказания, в то время как остальные алгоритмы обрушали сессию за недостатком памяти на меньшем объеме данных. Кроме того предсказания SVD модели оказались наиболее постоянными, стандартное отклонение у RMSE по 5 фолдам всего 0.0005.